In [132]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Input, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
import re
import os

In [2]:
data = pd.read_csv(os.path.join(os.getcwd(),"data/train.csv"),encoding='latin-1')
data.columns = ["polarity","id","date","query","user","text"]

stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]
data['text'] = data['text'].apply(lambda x: x.lower())
# def cleanupat(string):
#     newstring = [word for word in string.split() if word[0]!='@']
#     return " ".join(newstring)
data['text'] = data['text'].apply(lambda x: " ".join([word for word in x.split() if word[0]!='@']))
data['text'] = data['text'].apply(lambda x: " ".join([word for word in x.split() if word not in (stopwords)]))
print(data[data['polarity']==0].size)
# print(data[data['polarity']==1].size)
# print(data[data['polarity']==2].size)
# print(data[data['polarity']==3].size)
print(data[data['polarity']==4].size)

4799994
4800000


In [17]:
max_features = 3000
max_len = 50
tokenizer = Tokenizer(num_words=max_features, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X, maxlen = max_len)
X.shape

(1599999, 50)

In [19]:
#len(tokenizer.word_index)
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,  730,   31,  477,  448, 2011,   32,  218,
        464, 2310,   85,   12,  190, 1084], dtype=int32)

In [67]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            word_to_vec_map[w_line[0]] = np.array(w_line[1:], dtype=np.float64)
    return word_to_vec_map

word_to_vec_map = read_glove_vector("glove.twitter.27B.50d.txt")

emb_matrix = np.zeros((len(tokenizer.word_index), 50))
for word, index in tokenizer.word_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index, :] = embedding_vector
        
embedding_layer = Embedding(input_dim=(len(tokenizer.word_index)), output_dim=50, 
                                input_length=max_len, weights = [emb_matrix], trainable=False)

In [13]:
help(type(embedding_layer))

Help on class Embedding in module keras.src.layers.core.embedding:

class Embedding(keras.src.engine.base_layer.Layer)
 |  Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None, sparse=False, **kwargs)
 |  
 |  Turns positive integers (indexes) into dense vectors of fixed size.
 |  
 |  e.g. `[[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]`
 |  
 |  This layer can only be used on positive integer inputs of a fixed range. The
 |  `tf.keras.layers.TextVectorization`, `tf.keras.layers.StringLookup`,
 |  and `tf.keras.layers.IntegerLookup` preprocessing layers can help prepare
 |  inputs for an `Embedding` layer.
 |  
 |  This layer accepts `tf.Tensor`, `tf.RaggedTensor` and `tf.SparseTensor`
 |  input.
 |  
 |  Example:
 |  
 |  >>> model = tf.keras.Sequential()
 |  >>> model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
 |  >>> # The model will take as

In [58]:
embeddings = embedding_layer(Input(X.shape[1]))
type(embeddings)
print(X.shape[1])

50


In [101]:
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_380626 (Embeddin  (None, 50, 128)           384000    
 g)                                                              
                                                                 
 spatial_dropout1d_9 (Spati  (None, 50, 128)           0         
 alDropout1D)                                                    
                                                                 
 lstm_65 (LSTM)              (None, 196)               254800    
                                                                 
 dense_25 (Dense)            (None, 2)                 394       
                                                                 
Total params: 639194 (2.44 MB)
Trainable params: 639194 (2.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [141]:
embed_dim = 128
lstm_out = 196

# model = Sequential()
# model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
# model.add(SpatialDropout1D(0.4))
# model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(2,activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])


# l = LSTM(128, return_sequences=True)(embeddings)

# l = Dropout(0.6)(l)

# l = LSTM(128, return_sequences=True)(l)

# l = Dropout(0.6)(l)

# l = LSTM(128)(l)

# l = Dense(1, activation='sigmoid')(l)

# theinput = keras.Input(shape = (None, 50))

# model = Model(inputs=theinput, outputs=l)


model = Sequential()
#model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
model.add(embedding_layer)
# forward_layer = LSTM(lstm_out, return_sequences=True)
# backward_layer = LSTM(lstm_out, activation='relu', return_sequences=True,
#                       go_backwards=True)
# model.add(Bidirectional(forward_layer, backward_layer=backward_layer))
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
# model.add(keras.layers.fullyConnectedLayer(10))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(2,activation = 'sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])



print(model.summary())

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_380621 (Embeddin  (None, 50, 50)            19031050  
 g)                                                              
                                                                 
 lstm_78 (LSTM)              (None, 196)               193648    
                                                                 
 dense_33 (Dense)            (None, 32)                6304      
                                                                 
 dense_34 (Dense)            (None, 2)                 66        
                                                                 
Total params: 19231068 (73.36 MB)
Trainable params: 200018 (781.32 KB)
Non-trainable params: 19031050 (72.60 MB)
_________________________________________________________________
None


In [142]:
Y = pd.get_dummies(data['polarity']).values
smallsize = 3000
X_small = X[np.random.choice(X.shape[0], smallsize, replace=False)]
Y_small = Y[np.random.choice(Y.shape[0], smallsize, replace=False)]
X_train, X_test, Y_train, Y_test = train_test_split(X_small,Y_small, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2010, 50) (2010, 2)
(990, 50) (990, 2)


In [143]:
batch_size = 64
model.fit(X_train, Y_train, epochs = 32, batch_size=batch_size, verbose = 2)

Epoch 1/32
32/32 - 6s - loss: 0.7004 - accuracy: 0.4940 - 6s/epoch - 179ms/step
Epoch 2/32
32/32 - 5s - loss: 0.6935 - accuracy: 0.5194 - 5s/epoch - 152ms/step
Epoch 3/32
32/32 - 5s - loss: 0.6900 - accuracy: 0.5269 - 5s/epoch - 158ms/step
Epoch 4/32
32/32 - 5s - loss: 0.6879 - accuracy: 0.5328 - 5s/epoch - 158ms/step
Epoch 5/32
32/32 - 5s - loss: 0.6819 - accuracy: 0.5741 - 5s/epoch - 156ms/step
Epoch 6/32
32/32 - 5s - loss: 0.6785 - accuracy: 0.5667 - 5s/epoch - 158ms/step
Epoch 7/32
32/32 - 5s - loss: 0.6754 - accuracy: 0.5910 - 5s/epoch - 165ms/step
Epoch 8/32
32/32 - 5s - loss: 0.6694 - accuracy: 0.5851 - 5s/epoch - 160ms/step
Epoch 9/32
32/32 - 5s - loss: 0.6610 - accuracy: 0.5975 - 5s/epoch - 157ms/step
Epoch 10/32
32/32 - 5s - loss: 0.6544 - accuracy: 0.6060 - 5s/epoch - 158ms/step
Epoch 11/32
32/32 - 5s - loss: 0.6383 - accuracy: 0.6348 - 5s/epoch - 160ms/step
Epoch 12/32
32/32 - 5s - loss: 0.6317 - accuracy: 0.6493 - 5s/epoch - 164ms/step
Epoch 13/32
32/32 - 5s - loss: 0.6134

In [146]:
validation_size = 1000
def chooserandom(matrix, n):
    return matrix[np.random.choice(matrix.shape[0],n,replace=False)]
# X_validate = chooserandom(X_test,validation_size)
# Y_validate = chooserandom(X_test,validation_size)
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]

score,acc = model.evaluate(X_validate, Y_validate, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

16/16 - 1s - loss: 1.2994 - accuracy: 0.5051 - 583ms/epoch - 36ms/step
score: 1.30
acc: 0.51


In [147]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 0)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 48.06517311608962 %
neg_acc 52.90581162324649 %


In [ ]:
sometestthing = tokenizer.texts_to_sequences(["Looks like you're having a good day"])
sometestthing = pad_sequences(sometestthing, maxlen=max_len, dtype='int32', value=0)
sometestthing.shape

In [148]:
testvalue = input().lower()
sometestthing = tokenizer.texts_to_sequences([testvalue])
sometestthing = pad_sequences(sometestthing, maxlen=max_len, dtype='int32', value=0)
sometestthing.shape
sentiment = model.predict(sometestthing, batch_size=1,verbose = 0)
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

i like u
positive


In [157]:
from profanity_check import predict, predict_prob
def maketest():
    testvalue = input("Input your prompt:").lower()
    sometestthing = tokenizer.texts_to_sequences([testvalue])
    sometestthing = pad_sequences(sometestthing, maxlen=max_len, dtype='int32', value=0)
    sometestthing.shape
    sentiment = model.predict(sometestthing, batch_size=1,verbose = 0)
    if(np.argmax(sentiment) == 0):
        result = "negative"
    elif (np.argmax(sentiment) == 1):
        result = "positive"
    if (predict_prob([testvalue])>0.6):
        result+=" and possible profanity"
    print(result)
maketest()
maketest()

Input your prompt:haha puppy go brrrr
positive
Input your prompt:awww you like puppies too
positive
